### ***RAG Model***<br>
This notebook implements the RAG system using GPT-3.5 for query answering over UMBC ISSS Data.<br>
***Key Components:***
* ***Embeddings:***<br>
<tb>Generated using the sentence-transformers/all-mpnet-base-v2 model.<br>
* ***Hybrid Retrieval:***<br>
FAISS for dense retrieval and BM25 for sparse keyword-based retrieval.<br>
Hybrid retrieval combines FAISS and BM25 for accuracy.<br>
* ***Generation:***<br>
GPT-3.5 processes top-ranked context to generate detailed and accurate responses.<br>
* ***Evaluation:***<br>
Tested on 20 FAQs with metrics like Cosine Similarity and BERTScore.

## Importing Libraries

In [1]:
!pip install openai==0.28
!pip install faiss-cpu rank-bm25 sentence-transformers
!pip install  bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.6 MB/s eta 0:00:00


## **Data Loading from Github**

In [2]:
import requests
import json

# GitHub repository details
repo_owner = "Sritejam"
repo_name = "datahub"
folder_path = "LLM/processed_documents"

# Function to fetch JSON files from GitHub API
def fetch_files_from_github_folder(owner, repo, folder):
    base_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{folder}"
    try:
        response = requests.get(base_url)
        response.raise_for_status()
        file_urls = []
        for item in response.json():
            if item['type'] == 'file' and item['name'].endswith('.json'):
                file_urls.append(item['download_url'])
        return file_urls
    except requests.exceptions.RequestException as e:
        print(f"Error fetching folder contents from GitHub: {e}")
        return []


# Fetch list of JSON files in the folder
document_files = fetch_files_from_github_folder("Sritejam", "datahub", "LLM/processed_documents")

# Initialize a list to store all text chunks
documents = []

def process_chunk(chunk):
    processed_text = []
    for key, value in chunk.items():
        if isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    section_info = "; ".join(f"{k}: {v}" for k, v in item.items())
                    processed_text.append(f"{key} - {section_info}")
        elif isinstance(value, dict):
            nested_info = "; ".join(f"{k}: {v}" for k, v in value.items())
            processed_text.append(f"{key} - {nested_info}")
        else:
            processed_text.append(f"{key}: {value}")

    return " | ".join(processed_text)


# FAQ URL
faq_url = "https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/FAQ.json"

# fetch and parse JSON from raw GitHub URLs
def fetch_json_from_github(raw_url):
    try:
        response = requests.get(raw_url)
        response.raise_for_status()  # Check for HTTP errors
        return response.json()  # Parse JSON content
    except requests.exceptions.RequestException as e:
        print(f"Error fetching file from GitHub: {raw_url}")
        print(f"Error message: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file: {raw_url}")
        print(f"Error message: {e}")
        return None



# Fetch and process each document file
for file_url in document_files:
    data = fetch_json_from_github(file_url)
    if data:
        chunk_text = data.get('ChunkText')
        if isinstance(chunk_text, dict):
            documents.append(process_chunk(chunk_text))
        elif isinstance(chunk_text, str):
            documents.append(chunk_text)
        else:
            print(f"Loaded URL: {file_url}")

# Check the number of documents loaded
print(f"Loaded {len(documents)} documents.")

# Load preprocessed FAQs
faqs = fetch_json_from_github(faq_url)
if faqs:
    print(f"Loaded {len(faqs)} FAQs.")
else:
    print("Failed to load FAQs.")



Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/About_OISS.docx_chunk0.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/Admitted_Student_Events_and_Orientation.docx_chunk0.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/Admitted_Student_Events_and_Orientation.docx_chunk1.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/Applying_to_UMBC.docx_chunk0.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/Applying_to_UMBC.docx_chunk1.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/CPT.docx_chunk0.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/CPT.docx_chunk1.json
Loaded URL: https://raw.githubusercontent.com/Sritejam/datahub/main/LLM/processed_documents/CPT.docx_chu

In [3]:
documents[0]

'about oiss umbc office of international students and scholars the office of international students and scholars oiss is dedicated to supporting umbcs international student and scholar community. the office offers a variety of programming and services to assist students and scholars in pursuit of their academic, personal, and professional goals.'


## **Embeddings:**
Generated using the sentence-transformers/all-mpnet-base-v2 model.

## **Improving Retrieval with FAISS**
1. Hybrid Retrieval (Sparse + Dense):
Combine FAISS with traditional search like BM25. Sparse retrieval methods, like BM25, provide better keyword matching, while FAISS provides context. You can then score documents from both and use a reranker to get optimal results.
al performance by expanding the coverage of relevant content.
4. Re-Ranking:
Re-rank the top-k documents returned by FAISS using a fine-tuned BERT model to make sure the most relevant document appears first.

In [4]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi

# Load models

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
bert_reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

print("Models loaded successfully!")


# Tokenize the documents for BM25
tokenized_documents = [doc.split(" ") for doc in documents]

# Initialize BM25
bm25 = BM25Okapi(tokenized_documents)

# Generate embeddings for FAISS with FlatL2
def prepare_flat_faiss_index(documents):
    embeddings = np.array([embedding_model.encode(text, convert_to_tensor=False) for text in documents]).astype("float32")
    embedding_dim = embeddings.shape[1]

    # Create a FlatL2 index
    index = faiss.IndexFlatL2(embedding_dim)  # L2 distance
    index.add(embeddings)  #
    print(f"Flat FAISS index built with {index.ntotal} vectors.")
    return index, embeddings

# Build FAISS FlatL2 index
faiss_index, document_embeddings = prepare_flat_faiss_index(documents)

# Map for document metadata retrieval
document_metadata = {i: {"Text": doc} for i, doc in enumerate(documents)}



# Hybrid Retrieval
def hybrid_retrieval(query, top_k=3):
    # Sparse Retrieval with BM25
    bm25_scores = bm25.get_scores(query.split(" "))
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:top_k]
    sparse_results = [{"Text": documents[idx], "BM25_Score": bm25_scores[idx]} for idx in top_bm25_indices]

    # Dense Retrieval with FAISS
    query_embedding = embedding_model.encode(query, convert_to_tensor=False).astype("float32").reshape(1, -1)
    distances, indices = faiss_index.search(query_embedding, top_k)
    dense_results = [
        {"Text": document_metadata[idx]["Text"], "FAISS_Distance": distances[0][i]}
        for i, idx in enumerate(indices[0])
    ]

    # Combine Sparse and Dense Results

    seen_texts = set()
    combined_results = []
    for result in sparse_results + dense_results:
        if result["Text"] not in seen_texts:
            combined_results.append(result)
            seen_texts.add(result["Text"])

    # Re-rank results using BERT
    rerank_inputs = [(query, result["Text"]) for result in combined_results]
    rerank_scores = bert_reranker.predict(rerank_inputs)
    for i, result in enumerate(combined_results):
        result["Rerank_Score"] = rerank_scores[i]
        # Calculate cosine similarity
        doc_embedding = embedding_model.encode(result["Text"], convert_to_tensor=False).astype("float32").reshape(1, -1)
        result["Cosine_Similarity"] = cosine_similarity(query_embedding, doc_embedding)[0][0]


    # Sort by Rerank_Score
    final_results = sorted(combined_results, key=lambda x: x["Rerank_Score"], reverse=True)[:top_k]
    return final_results




/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Models loaded successfully!
Flat FAISS index built with 493 vectors.


## **Using GPT-3.5 as Generator**

In [5]:
from bert_score import score
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [6]:
# set up OpenAI
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [7]:
# GPT-3.5 Generator Function
def generate_response(input_text, retrieved_context, max_length=100):
    #  message for GPT
    messages1 = [
        {"role": "system", "content": "You are a professional assistant who provides concise and formal answers."},
        {"role": "system", "content": f"Use the following context to answer the question:\n{retrieved_context}"},
        {"role": "user", "content": input_text}
    ]

    # OpenAI API call
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages1,
        max_tokens=max_length,
        temperature=0.2
    )

    return response['choices'][0]['message']['content'].strip()

# Combine retrieved documents into context for the generator
def generate_response_with_retrieved_docs(query, retrieved_docs, top_k=3):
    retrieved_context = "\n".join([f"Context: {doc['Text']}" for doc in retrieved_docs[:top_k]])
    input_text = f"Answer the following query: {query}"
    #print(f"Input to GPT-3.5 model:\n{input_text[:750]}...\n")  # Print the first 750 characters of the input.
    generated_answer = generate_response(input_text, retrieved_context)
    return generated_answer


In [8]:
# Example: Run the hybrid retrieval and use generator for response
query = "Do I need a job before applying for STEM OPT?"

# Perform hybrid retrieval
retrieved_docs = hybrid_retrieval(query, top_k=3)

# Generate a detailed answer based on retrieved results
generated_answer = generate_response_with_retrieved_docs(query, retrieved_docs, top_k=3)

print("\nGenerated Answer:\n", generated_answer)



Generated Answer:
 Yes, you need to have a job offer from an employer enrolled and participating in the E-Verify program before applying for STEM OPT.


In [9]:
# Example: Run the hybrid retrieval and use generator for response
query = "When is Large Language Models course next semester at UMBC? Give getails about it."

# Perform hybrid retrieval
retrieved_docs = hybrid_retrieval(query, top_k=3)

# Generate a detailed answer based on retrieved results
generated_answer = generate_response_with_retrieved_docs(query, retrieved_docs, top_k=3)

print("\nGenerated Answer:\n", generated_answer)


Generated Answer:
 The Large Language Models course (Course Code: DATA 690) at UMBC is scheduled for next semester. The course is offered in Section 08 on Tuesdays from 7:10 PM to 9:40 PM in Fine Arts 001. The instructor is Donghwa Kim. It is a 3-unit course with 25 total seats, of which 17 are currently available. The instruction mode is hybrid.


## **Evaluation**

In [10]:
def evaluate_all_faqs_with_mean(faqs, top_k=3):
    results = []

    for idx, faq in enumerate(faqs):  # Iterate over all FAQs
        query = faq["question"]
        gold_answer = faq["answer"]
        print(f"\nProcessing FAQ #{idx + 1}")
        print(f"Query: {query}")

        # Step 1: Retrieve context
        retrieved_results = hybrid_retrieval(query, top_k=top_k)

        # Extract cosine similarities
        cosine_similarities = [result["Cosine_Similarity"] for result in retrieved_results]

        # Compute retrieval metric (mean similarity score)
        retrieval_score = np.mean(cosine_similarities)
        print(f"Retrieval Metric (Mean Cosine Similarity): {retrieval_score:.4f}")

        # Use the top retrieved context for generation
        retrieved_context = " ".join([result["Text"] for result in retrieved_results])

        # Debug: Retrieved results with cosine similarities
        print("Top Retrieved Results with Cosine Similarities:")
        for i, (text, cos_sim) in enumerate(zip([r["Text"] for r in retrieved_results], cosine_similarities)):
            print(f"Top-{i+1} Text: {text}\nCosine Similarity: {cos_sim:.4f}\n")

        # Step 2: Generate response using the retrieved context
        generated_response = generate_response(input_text=query, retrieved_context=retrieved_context)

        # Debug: Generated response
        print(f"Generated Response:\n{generated_response}")

        # Step 3: Evaluate the generated response against the gold answer
        P, R, F1 = score([generated_response], [gold_answer], lang="en", verbose=False)

        # Debug: BERTScore metrics
        print(f"BERTScore - Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")

        # Collect results
        results.append({
            "query": query,
            "gold_answer": gold_answer,
            "retrieved_context": retrieved_context,
            "generated_response": generated_response,
            "retrieval_score": retrieval_score,  # Mean cosine similarity
            "bertscore_precision": P.mean().item(),
            "bertscore_recall": R.mean().item(),
            "bertscore_f1": F1.mean().item(),
        })

    # Convert results to DataFrame for better visualization
    results_df = pd.DataFrame(results)

    # Compute mean scores for each metric
    mean_retrieval_score = results_df["retrieval_score"].mean()
    mean_precision = results_df["bertscore_precision"].mean()
    mean_recall = results_df["bertscore_recall"].mean()
    mean_f1 = results_df["bertscore_f1"].mean()

    # Print mean scores
    print("\nMean Scores Across All 20 FAQs:")
    print(f"Mean Retrieval Score (Cosine Similarity): {mean_retrieval_score:.4f}")
    print(f"Mean BERTScore Precision: {mean_precision:.4f}")
    print(f"Mean BERTScore Recall: {mean_recall:.4f}")
    print(f"Mean BERTScore F1: {mean_f1:.4f}")

    return results_df, {
        "mean_retrieval_score": mean_retrieval_score,
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1": mean_f1,
    }


## **Results**

In [11]:
# Run the evaluation and get results
faq_results, mean_scores = evaluate_all_faqs_with_mean(faqs, top_k=3)


# Display mean scores
print("\nMean Scores:")
print(mean_scores)



Processing FAQ #1
Query: What services does the ISSS office provide?
Retrieval Metric (Mean Cosine Similarity): 0.4759
Top Retrieved Results with Cosine Similarities:
Top-1 Text: the office offers a variety of programming and services to assist students and scholars in pursuit of their academic, personal, and professional goals.
Cosine Similarity: 0.4582

Top-2 Text: isss office closed on thursday, november 7 if you have a question or concern, please submit a help ticket below and our team will respond as soon as we can. thank you! help ticket the isss office does not respond to emails. instead, we use a help ticket system to send and receive messages.
Cosine Similarity: 0.5398

Top-3 Text: about oiss umbc office of international students and scholars the office of international students and scholars oiss is dedicated to supporting umbcs international student and scholar community. the office offers a variety of programming and services to assist students and scholars in pursuit of th

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

BERTScore - Precision: 0.9008, Recall: 0.8991, F1: 0.8999

Processing FAQ #19
Query: How do I get a Social Security Number (SSN)?
Retrieval Metric (Mean Cosine Similarity): 0.6128
Top Retrieved Results with Cosine Similarities:
Top-1 Text: ssn are issued by the united states social security administration ssa. to start the process, students should submit a ssn letter request in the isss portal. after a student has an I-20 with cpt approval and a receipt from the ssa, they present these documents to their employers human resources department.
Cosine Similarity: 0.6255

Top-2 Text: applying for a social security number ssn an ssn is required of any individual in order to get paid in the us.
Cosine Similarity: 0.6088

Top-3 Text: when you receive your first job offer, you will need to apply for a social security number ssn if you dont already have one. an ssn is required to be paid in the us. to learn more about what an ssn is and how to apply, please review the information in this websit

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.8383, Recall: 0.8806, F1: 0.8589

Processing FAQ #20
Query: What should I do if my I-20 is about to expire?
Retrieval Metric (Mean Cosine Similarity): 0.6087
Top Retrieved Results with Cosine Similarities:
Top-1 Text: deadline please note that your form I-20 can only be extended up until the current end date. after your current I-20 end date passes, we can no longer extend the date into the future. it is important to plan ahead for this deadline.
Cosine Similarity: 0.5679

Top-2 Text: please note, you must work with us to get your new I-20 before your current I-20 end date, plus the 60 day grace period.
Cosine Similarity: 0.6465

Top-3 Text: step 3: request an I-20 reprint if needed you will need an updated I-20 if: your employer requests it from you you need to renew your drivers license you plan on travelling abroad in the next 6 months note: students on OPT will not be able to access the isss portal using their myumbc credentials.
Cosine Similarity: 0.6116



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.8523, Recall: 0.8815, F1: 0.8666

Mean Scores Across All 20 FAQs:
Mean Retrieval Score (Cosine Similarity): 0.5194
Mean BERTScore Precision: 0.8649
Mean BERTScore Recall: 0.8915
Mean BERTScore F1: 0.8778

Mean Scores:
{'mean_retrieval_score': 0.51935446, 'mean_precision': 0.8648564994335175, 'mean_recall': 0.891491037607193, 'mean_f1': 0.8777661472558975}


In [12]:
# Display mean scores
print("\nMean Scores:")
print(mean_scores)


Mean Scores:
{'mean_retrieval_score': 0.51935446, 'mean_precision': 0.8648564994335175, 'mean_recall': 0.891491037607193, 'mean_f1': 0.8777661472558975}
